This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'some api key'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2019-04-29&end_date=2019-04-29&api_key=" + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-05-01T22:34:46.890Z',
  'newest_available_date': '2019-04-30',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-04-29',
  'end_date': '2019-04-29',
  'data': [['2019-04-29',
    None,
    87.65,
    85.9,
    87.5,
    None,
    150210.0,
    13060972.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# TASK 1

In [5]:
# Collecting the data from Franfurt Stock Exchange, for ticker AFX_X for the year 2017

afx_x_2017 = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY)

# TASK 2

In [6]:
# Converting the returned JSON object to Python Dictionary

afx_x_17_dict = json.loads(afx_x_2017.text)
type(afx_x_17_dict)

dict

# TASK 3

In [7]:
# Calculate highest and lowest opening price of the stock in 2017

opening_prices = [daily[1] for daily in afx_x_17_dict['dataset']['data'] if daily[1] is not None]
min_opening = min(opening_prices)
max_opening = max(opening_prices)
print('Opening Prices for the period 2017: ' + 'Lowest Price is $' + str(min_opening) + ' & Highest Price is $' + str(max_opening))

Opening Prices for the period 2017: Lowest Price is $34.0 & Highest Price is $53.11


# TASK 4

In [8]:
# Largest change in any day(based on High and Low Price)

anyday_change = [(daily[2] - daily[3]) for daily in afx_x_17_dict['dataset']['data']]
print('Largest change in any day based on High and Low price is:' + ' $' + str(max(anyday_change)))

Largest change in any day based on High and Low price is: $2.8100000000000023


# TASK 5

In [9]:
# Largest change between any two days(based on Closing price)

two_day_change = []
last_day_change = 0

for daily in afx_x_17_dict['dataset']['data']:
    if last_day_change == 0:
        last_day_change = daily[4]
    else:
        two_day_change.append(daily[4] - last_day_change)
        last_day_change = daily[4]
        
max_change = max(two_day_change)
min_change = min(two_day_change)

if abs(min_change) > max_change:
    print ('Largest change between any two days based on Closing price is:' + ' $' + str(abs(min_change)))
else:
    print('Largest change between any two days based on Closing price is:' + ' $' + str(max_change))

Largest change between any two days based on Closing price is: $2.559999999999995


# TASK 6

In [10]:
# Average trading volume in 2017

total_vol = 0
for daily in afx_x_17_dict['dataset']['data']:
    total_vol += daily[6]
avg_vol = total_vol/len(afx_x_17_dict['dataset']['data'])
print('The Average volume during this year is: ' + str(round(avg_vol)))

The Average volume during this year is: 89124


# TASK 7

In [11]:
# Median trading volume during this year

trade_vol = sorted([daily[6] for daily in afx_x_17_dict['dataset']['data']])
trade_len = len(trade_vol)
half_len = trade_len/2

if trade_len%2 == 1:
    print('Median trading volume during this year is: ' + str(trade_vol[round(half_len)]))
else:
    print('Median trading volume during this year is: ' + str(trade_vol[half_len]+trade_vol[half_len+1])/2)

Median trading volume during this year is: 76600.0
